In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# imports

In [58]:
import torch
import gym
from firefly_task import Model as Task
from stable_baselines3 import PPO
from ult import *
import os
import sys
sys.path.append(f'{os.getcwd()}')
import warnings
warnings.filterwarnings("ignore")

# prepare

In [59]:
modelname = 'movingff_test0908'

epoch_size = 33333
n_epoch = 33
n_seed = 1
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

# for plotting
num_eval_episode = 2222  # for eval plot during training



# train

In [60]:
for iepoch in range(0, n_epoch):
    # training ----------------
    thismodel = f'seed{seed}_model{modelname}_ep{iepoch}'

    env=Task(dt=1)
    env.debug=0
    # train
    if iepoch == 0:
        model = PPO("MlpPolicy", env, verbose=0, device='cpu',clip_range=0.1, ent_coef=0.01)

    else:
        previous_model = f'seed{seed}_model{modelname}_ep{iepoch-1}'
        model = PPO.load(f'ycstore/{previous_model}', env=env)
    model.learn(total_timesteps=epoch_size,)
    model.save(f'ycstore/{thismodel}')
    print(f'epoch{iepoch}, yctore/{thismodel} saved')


    # eval collection --------------------
    results=[]
    thismodel = f'seed{seed}_model{modelname}_ep{iepoch}'
    model = PPO.load(f'ycstore/{thismodel}')

    def eval_wrapper(a):
        episode = run_one_episode(env, model, deterministic=True)
        return episode

    with multiprocess.Pool(processes=8) as pool:
        all_episode_data = pool.map(eval_wrapper, range(num_eval_episode))
    
    for episode in all_episode_data:
        # process
        episode['sum_reward']=sum(episode['rewards'])
        results.append(episode)

    df = pd.DataFrame(results)
    print(f'avg reward per eval trials: {sum(np.concatenate(df.rewards.to_numpy()))/num_eval_episode:.2f}')
    # eval_plot(df)



epoch0, yctore/seed0_modelmovingff_test0908_ep0 saved
avg reward per eval trials: 0.58
epoch1, yctore/seed0_modelmovingff_test0908_ep1 saved
avg reward per eval trials: 0.64
epoch2, yctore/seed0_modelmovingff_test0908_ep2 saved
avg reward per eval trials: 0.60
epoch3, yctore/seed0_modelmovingff_test0908_ep3 saved
avg reward per eval trials: 0.58
epoch4, yctore/seed0_modelmovingff_test0908_ep4 saved
avg reward per eval trials: 0.57
epoch5, yctore/seed0_modelmovingff_test0908_ep5 saved
avg reward per eval trials: 0.64
epoch6, yctore/seed0_modelmovingff_test0908_ep6 saved
avg reward per eval trials: 0.57
epoch7, yctore/seed0_modelmovingff_test0908_ep7 saved
avg reward per eval trials: 0.67
epoch8, yctore/seed0_modelmovingff_test0908_ep8 saved
avg reward per eval trials: 0.69
epoch9, yctore/seed0_modelmovingff_test0908_ep9 saved
avg reward per eval trials: 0.66
epoch10, yctore/seed0_modelmovingff_test0908_ep10 saved
avg reward per eval trials: 0.64
epoch11, yctore/seed0_modelmovingff_test0

KeyboardInterrupt: 

In [ ]:
belief=env.reset()
f, v, w, sig_ff, sig_vv, sig_fv=belief
f, v, w, sig_ff, sig_vv, sig_fv
f,v,w=env.x
f,v,w

(tensor(4.7860), tensor([0.7515]), tensor([0.]))

In [ ]:
# check the spaces of variables.
# note that (action, obs) are shorter than states by 1. they are 'transitions'
env.timeout=22
ep=run_one_episode(env, model, deterministic=True)
for k,v in ep.items():
    try:
        print(f'{k}, {v.shape}')
    except:
        print(f'{k},{v}')

[-0.012117770500481129, -0.016134459525346756, -0.021752431988716125, -0.02747291326522827, -0.032454680651426315, -0.03550109639763832, -0.03493702411651611, -0.02975146844983101, -0.019988559186458588, -0.006651879288256168, 0.00864340364933014, 0.024282068014144897, 0.039094362407922745, 0.05243770405650139, 0.06407687813043594, 0.0740325003862381, 0.08245997130870819, 0.08956790715456009, 0.09556984156370163, 0.10065941512584686, 0.1050000712275505, 0.10872343182563782, 0.1119321957230568, 0.11470527201890945]
num_steps,24
actions, (24,)
rewards, (24,)
states, (25, 3)
observations, (24, 1)
beliefs, (25, 3)
obs_phase_f, torch.Size([10])
obs_phase_w, torch.Size([10])
